# Brain-Supervised Image Editing
## Supplementary Material
### Code

# EEG Data Prep - To Be Deleted

In [1]:
import numpy as np
import scipy as sp
import mne
from os import listdir
from os.path import join
import pandas as pd
import sklearn
import re
import pickle
from tqdm.notebook import tqdm
import sys

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
#from neuromancer import utils
epodir = "../data/EEG_New/data/cleaned/"
from functools import reduce
import matplotlib
matplotlib.use('qt5agg') # Replace with appropriate backend
import matplotlib.pyplot as plt
import mne
import pandas as pd
from os import listdir, path

TASK_LIST = ['facecat/female','facecat/male',
             'facecat/nosmile', 'facecat/smiles',
             'facecat/old', 'facecat/young',
            'facecat/blond', 'facecat/darkhaired']

# Load epochs and drop EOG data

In [2]:
epos = []
for f in sorted(listdir(epodir)):
    epo = mne.read_epochs(path.join(epodir, f))
    # Drop the EOG channels
    epos.append(epo.drop_channels(["HEOG", "VEOG"]))

Reading ../data/EEG_New/data/cleaned/P01-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     900.00 ms
        0 CTF compensation matrices available
Adding metadata with 3 columns
Replacing existing metadata with 3 columns
3342 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Reading ../data/EEG_New/data/cleaned/P02-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     900.00 ms
        0 CTF compensation matrices available
Adding metadata with 3 columns
Replacing existing metadata with 3 columns
3444 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Reading ../data/EEG_New/data/cleaned/P03-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     900.00 ms
        0 CTF compensation matrices available
Adding metadata with 3 columns
Replacing existing metadata with 3 columns
3203 matching events found
Applying baseline correction (mode: mea

0 projection items activated
Reading ../data/EEG_New/data/cleaned/P25-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     900.00 ms
        0 CTF compensation matrices available
Adding metadata with 3 columns
Replacing existing metadata with 3 columns
3396 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Reading ../data/EEG_New/data/cleaned/P26-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     900.00 ms
        0 CTF compensation matrices available
Adding metadata with 3 columns
Replacing existing metadata with 3 columns
2874 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Reading ../data/EEG_New/data/cleaned/P27-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     900.00 ms
        0 CTF compensation matrices available
Adding metadata with 3 columns
Replacing existing metadata with 3 columns
3426 matching events found
Applying b

In [3]:
epos[0]['facecat/blond'].get_data().shape

(253, 32, 551)

In [4]:
def epoch_to_lda_data(epoch, times=[0, 0.5]):
    
    
    # Set up task dictionary to identify test/train splits
    
    task_dict = {'facecat/blond':['facecat/female','facecat/male', 
                                  'facecat/nosmile', 'facecat/old', 
                                  'facecat/smiles','facecat/young'],
                 'facecat/darkhaired':['facecat/female', 'facecat/male', 
                                       'facecat/nosmile', 'facecat/old', 
                                        'facecat/smiles','facecat/young'], 
                 'facecat/female': ['facecat/blond','facecat/darkhaired',
                                    'facecat/nosmile', 'facecat/old', 
                                    'facecat/smiles','facecat/young'],
                 'facecat/male':['facecat/blond','facecat/darkhaired',
                                 'facecat/nosmile', 'facecat/old', 
                                 'facecat/smiles','facecat/young'], 
                 'facecat/nosmile':['facecat/blond','facecat/darkhaired',
                                     'facecat/female', 'facecat/male',
                                     'facecat/old','facecat/young'],
                 'facecat/old':['facecat/blond','facecat/darkhaired',
         'facecat/female', 'facecat/male', 'facecat/nosmile', 
         'facecat/smiles'],
                 'facecat/smiles':['facecat/blond','facecat/darkhaired',
         'facecat/female', 'facecat/male', 'facecat/old', 'facecat/young'],
                 'facecat/young':['facecat/blond','facecat/darkhaired',
         'facecat/female', 'facecat/male', 'facecat/nosmile',
         'facecat/smiles']}
    
    
    # Get time indexes for slicing
    tr_times = epoch.time_as_index([times[0], times[1]])
    
    lda_data = {}
    
    # Create train/test splits
    for task in task_dict:
        
        train_x = []
        train_y = []
        
        for other_task in task_dict[task]:
            train_x.extend(np.array(epoch[other_task].get_data())[:,:,tr_times[0]:tr_times[1]])
            train_y.append(epoch[other_task].metadata)
        
        lda_data[task] = {"test_x": np.array(epoch[task].get_data())[:,:,tr_times[0]:tr_times[1]] , 
                  "test_y": epoch[task].metadata.reset_index(drop=True), 
                  "train_x": np.array(train_x), 
                  "train_y": pd.concat(train_y).reset_index(drop=True)}
    
    return lda_data
    

In [5]:
    task_dict = {'facecat/blond':['facecat/female','facecat/male', 
                                  'facecat/nosmile', 'facecat/old', 
                                  'facecat/smiles','facecat/young'],
                 'facecat/darkhaired':['facecat/female', 'facecat/male', 
                                       'facecat/nosmile', 'facecat/old', 
                                        'facecat/smiles','facecat/young'], 
                 'facecat/female': ['facecat/blond','facecat/darkhaired',
                                    'facecat/nosmile', 'facecat/old', 
                                    'facecat/smiles','facecat/young'],
                 'facecat/male':['facecat/blond','facecat/darkhaired',
                                 'facecat/nosmile', 'facecat/old', 
                                 'facecat/smiles','facecat/young'], 
                 'facecat/nosmile':['facecat/blond','facecat/darkhaired',
                                     'facecat/female', 'facecat/male',
                                     'facecat/old','facecat/young'],
                 'facecat/old':['facecat/blond','facecat/darkhaired',
         'facecat/female', 'facecat/male', 'facecat/nosmile', 
         'facecat/smiles'],
                 'facecat/smiles':['facecat/blond','facecat/darkhaired',
         'facecat/female', 'facecat/male', 'facecat/old', 'facecat/young'],
                 'facecat/young':['facecat/blond','facecat/darkhaired',
         'facecat/female', 'facecat/male', 'facecat/nosmile',
         'facecat/smiles']}

In [6]:
list(task_dict.keys())

['facecat/blond',
 'facecat/darkhaired',
 'facecat/female',
 'facecat/male',
 'facecat/nosmile',
 'facecat/old',
 'facecat/smiles',
 'facecat/young']

In [7]:
def split_average(df, n_split=15):
    """
    Take epoch data and average into N_Splits,
    n_split:  The SMALLER the number, the lower resolution the data is
    """
    splits = np.split(df, n_split, axis=2)
    mean_list = []
    for i in splits:
        mean_list.append(np.mean(i, axis=2))
    
    return(np.stack(mean_list, axis=1))

In [8]:
output_data = dict()
for i in tqdm(range(30)):
    output_data[i] = dict()
    for task in tqdm(list(task_dict.keys()), leave=False):
        output_data[i][task] = dict()
        lda_data = epoch_to_lda_data(epos[i], times=[0, 0.5])
        try:
            test_x = split_average(lda_data[task]['test_x'], n_split=5)
            train_x = split_average(lda_data[task]['train_x'], n_split=5)
            
            output_data[i][task]['test'] = test_x
            output_data[i][task]['train'] = train_x
        except:
            pass
                                    

In [10]:
import lzma
# open a file, where you ant to store the data
file = lzma.open('facecat.xz', 'wb')

# dump information to that file
pickle.dump(output_data, file)

# close the file
file.close()

In [ ]:
lda_data = epoch_to_lda_data(epos[2], times=[0, 0.5])

In [ ]:
test_x = split_average(lda_data['facecat/blond']['test_x'], n_split=5)
train_x = split_average(lda_data['facecat/blond']['train_x'], n_split=5)

In [ ]:
lda_data

In [ ]:
# open a file, where you ant to store the data
file = open('facecat.pkl', 'wb')

# dump information to that file
pickle.dump(lda_data, file)

# close the file
file.close()

In [ ]:
test_y = lda_data['facecat/blond']['test_y'].rel
train_y = lda_data['facecat/blond']['train_y'].rel

In [ ]:
text_x

In [ ]:
train_x = train_x.reshape([train_x.shape[0], train_x.shape[1] * train_x.shape[2]]) 

In [ ]:
lda = LDA(solver = 'lsqr', shrinkage = 'auto')
lda = lda.fit(train_x, list(train_y)) 

In [ ]:
test_x = test_x.reshape([test_x.shape[0], test_x.shape[1] * test_x.shape[2]])

# Reshape Data so it is anonymized (100ms averages), with channel names

In [ ]:
df

In [ ]:
df_example = epos[0]['facecat/blond']

In [ ]:
epos[0].ch_names

In [ ]:
epos[0]['facecat/blond'].pick_channels(['Fp1']).get_data().shape

In [ ]:
epos[0]['facecat/blond'].get_data()

In [ ]:
yhat = lda.predict_proba(test_x)

In [ ]:
sklearn.metrics.roc_auc_score(list(test_y), yhat[:,1], average="macro", multi_class='ovo')

In [ ]:
 # Results Loop
results_dict = {}
for user in tqdm(range(len(epos))):
    results_dict[user] = {}
    lda_data = epoch_to_lda_data(epos[user], times=[-0.2, 0.9])
    
    for task in TASK_LIST:
        test_x = split_average(lda_data[task]['test_x'], n_split=25)
        train_x = split_average(lda_data[task]['train_x'], n_split=25)
        
        train_x = train_x.reshape([train_x.shape[0], train_x.shape[1] * train_x.shape[2]])

        test_y = lda_data[task]['test_y'].rel
        train_y = lda_data[task]['train_y'].rel
        
        img_ids = lda_data[task]['test_y'].img
        event_type =  lda_data[task]['test_y'].event
        
        lda = LDA(solver = 'lsqr', shrinkage = 'auto')
        lda = lda.fit(train_x, list(train_y)) 
        
        test_x = test_x.reshape([test_x.shape[0], test_x.shape[1] * test_x.shape[2]])
        yhat = lda.predict_proba(test_x)[:,1]
        
        results_dict[user][task] = {"yhat":yhat, "y":test_y, "img_id": list(img_ids), "event":event_type}

In [ ]:
sklearn.metrics.roc_auc_score(list(results_dict[0]['facecat/female']['y']), results_dict[0]['facecat/female']['yhat'], average="macro", multi_class='ovo')

In [ ]:
 # Results Loop
loresresults_dict = {}
for user in tqdm(range(len(epos))):
    loresresults_dict[user] = {}
    lda_data = epoch_to_lda_data(epos[user], times=[0, 0.5])
    
    for task in TASK_LIST:
        test_x = split_average(lda_data[task]['test_x'], n_split=5)
        train_x = split_average(lda_data[task]['train_x'], n_split=5)
        
        train_x = train_x.reshape([train_x.shape[0], train_x.shape[1] * train_x.shape[2]])

        test_y = lda_data[task]['test_y'].rel
        train_y = lda_data[task]['train_y'].rel
        
        img_ids = lda_data[task]['test_y'].img
        event_type =  lda_data[task]['test_y'].event
        
        lda = LDA(solver = 'lsqr', shrinkage = 'auto')
        lda = lda.fit(train_x, list(train_y)) 
        
        test_x = test_x.reshape([test_x.shape[0], test_x.shape[1] * test_x.shape[2]])
        yhat = lda.predict_proba(test_x)[:,1]
        
        loresresults_dict[user][task] = {"yhat":yhat, "y":test_y, "img_id": list(img_ids), "event":event_type}

In [ ]:
sklearn.metrics.roc_auc_score(list(loresresults_dict[0]['facecat/female']['y']), loresresults_dict[0]['facecat/female']['yhat'], average="macro", multi_class='ovo')

# EEG Data - 100ms Resolution

# Stimuli Data

# SVR

# GAN Warping

# Github Account